# Team Model Mavericks: Text Wrangling Part of the event

In [ ]:
# For Google Colab connected to a Google Drive Only
# data will be deleted from google drive after 
# Model Mavericks: Kush, Nikki, Quinn, Clara, Rose
! cd ..
! cp "/content/drive/MyDrive/Colab Notebooks/dataFest.zip" "/content/sample_data/"
!unzip /content/sample_data/dataFest.zip >> out.txt
!mv "/content/DataFest 2023 Data For Distribution/data" "/"
!sudo apt install neofetch >> out.txt
!ls -la /data
!neofetch --stdout

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from collections import Counter
nltk.download('stopwords')
nltk.download('punkt')
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

<hr>

In [ ]:
# Todo: change the relevant portions if copying to a new Jupyter Notebook
# Reading in all the (relevant) CSVs
attorneys = pd.read_csv('/data/attorneys.csv')
attorneytimeentries = pd.read_csv('/data/attorneytimeentries.csv')
clients = pd.read_csv('/data/clients.csv')
categories = pd.read_csv('/data/categories.csv')
questions = pd.read_csv('/data/questions.csv')
#questionposts = pd.read_csv('/data/questionposts.csv')
statesites = pd.read_csv('/data/statesites.csv')
subcategories = pd.read_csv('/data/subcategories.csv')

In [ ]:
with open('/data/questionposts.csv', 'rb') as f:
    data = f.read()
    # Replace any NULL bytes with the string "<NULL>"
    data = data.replace(b'\x00', b'<NULL>')
    # Decode the data using the utf-8 encoding
    decoded_data = data.decode('utf-8')

# Now you can parse the CSV data as before
reader = csv.reader(decoded_data.splitlines(), delimiter=',', quotechar='"')
rows = []
for cols in reader:
    # Extract the ID, state, question ID, and date fields
    if len(cols) < 4:
        continue
    id_num = cols[0]
    state = cols[1]
    question_id = cols[2]
    post_text = ','.join(cols[3:-1]).replace('<NULL>', '\x00')
    created_utc = cols[-1]
    # Append the row data as a tuple to the list of rows
    rows.append((id_num, state, question_id, post_text, created_utc))

# Create a dataframe from the list of rows
questionposts = pd.DataFrame(rows, columns=['ID', 'StateAbbr', 'QuestionUno', 'PostText', 'CreatedUtc'])
questionposts = questionposts.drop(0) # The first row is literally just names of columns
questionposts = questionposts.reset_index(drop=True) # Do not need the first index column
questionposts = questionposts.set_index("ID") # ID is the index, so resetting back to it

In [ ]:
questionposts, questionposts.columns, questions.columns

In [ ]:
merged_expanded_questions = pd.merge(questions, questionposts, on="QuestionUno")
merged_expanded_questions = merged_expanded_questions.rename(columns={'StateAbbr_x': 'StateAbbr'})
merged_expanded_questions = merged_expanded_questions.drop('StateAbbr_y', axis=1)

In [ ]:
merged_expanded_questions.columns

In [ ]:
merged_expanded_questions

In [ ]:
# Finding Top Unique Questions Per Category 
# By question ID
# Group the dataframe by category and question ID and count the occurrences
question_count = merged_expanded_questions.groupby(['Category', 'QuestionUno']).size().reset_index(name='Count')

# Sort the dataframe by count in descending order
question_count = question_count.sort_values(by=['Category', 'Count'], ascending=[True, False])

# Display the top 10 questions per category
for category in question_count['Category'].unique():
    print(f'\nTop 10 questions in {category}:')
    print(question_count[question_count['Category']==category].head(10))

In [ ]:
# Group by category and subcategory
grouped = merged_expanded_questions.groupby(['Category', 'Subcategory'])

# Initialize the list of stopwords
stop_words = stopwords.words('english') + list(string.punctuation) + ['#']
stemmer = SnowballStemmer('english')

In [ ]:
# Group by state abbreviation, category, and subcategory
grouped = merged_expanded_questions.groupby(['StateAbbr', 'Category', 'Subcategory'])

# Count the number of questions in each group
counts = grouped.size()

# Convert the counts series to a dataframe and reset the index
counts_df = counts.to_frame(name='Question Count').reset_index()

# Sort the dataframe by state abbreviation and question count
highest_counts = counts_df.sort_values(['StateAbbr', 'Question Count'], ascending=[True, False])

# Keep only the highest counts for each state abbreviation
highest_counts = highest_counts.groupby('StateAbbr').head(3)

In [ ]:
#highest_categories = highest_categories.sort_values(by = "QuestionUno", ascending = False)
highest_categories

In [ ]:
merged_df = highest_categories.drop('Subcategory', axis=1)
merged_df = highest_categories.groupby(['StateAbbr', 'Category'])['QuestionUno'].sum().reset_index()

# drop the SubCategory column


# sort the dataframe by StateAbbr and Category
merged_df = merged_df.sort_values(['StateAbbr', 'Category'])

# reset the index
merged_df = merged_df.reset_index(drop=True)

In [ ]:
merged_df_summed = merged_df.groupby(['StateAbbr', 'Category'])['QuestionUno'].sum().reset_index()
merged_df_summed

In [ ]:
merged_df_summed.to_csv("Highest_Categories_No_Subcategories.csv")

In [ ]:
highest_categories.to_csv("Highest Categories")

In [ ]:
df = merged_expanded_questions
train, test = train_test_split(df, test_size=0.005)

In [ ]:
# Load the dataframe
df = test

# Define the stemmer and stopwords
stemmer = SnowballStemmer('english')
#stop_words = stopwords.words('english')
stop_words += ['thank', 'thanks', 'greetings']

# Create an empty dataframe to store the results
results = pd.DataFrame(columns=['State', 'Category', 'Top Terms', 'Top Questions'])

# Group by state, category, and subcategory
grouped = df.groupby(['StateAbbr', 'Category', 'Subcategory'])

# Loop through each group
for name, group in grouped:
    state = name[0]
    category = name[1] + ' - ' + name[2]

    # Tokenize the questions and remove stopwords
    tokens = [word.lower() for sentence in group['PostText'] for word in word_tokenize(sentence) if word.lower() not in stop_words and not word.isdigit() and len(word) > 1]

    # Count the frequency of each term or phrase
    frequency = Counter(tokens)

    # Get the top 10 terms
    top_terms = [t for t, f in frequency.most_common(10)]

    # Get the top 10 questions
    top_questions = []
    for post in group['PostText']:
        sentences = sent_tokenize(post)
        for sentence in sentences:
            if any(q.lower() in sentence.lower() for q in top_terms):
                top_questions.append(sentence.strip())

    # Add the results to the dataframe
    results = results.append({'State': state, 'Category': category, 'Top Terms': ', '.join(top_terms), 'Top Questions': '\n\n'.join(top_questions)}, ignore_index=True)

# Sort the results by state and question frequency
results = results.sort_values(by=['State', 'Top Questions'], ascending=[True, False])

In [ ]:
results

In [ ]:
# Load the dataframe
df = test

# Initialize the stemmer and stopwords
#stemmer = SnowballStemmer('english')
#stop_words = stopwords.words('english')

# Create a new dataframe to store the results
results_df = pd.DataFrame(columns=['State', 'Category', 'Top Terms', 'Top Questions'])

# Group by state and category
grouped = df.groupby(['StateAbbr', 'Category'])

# Loop through each group
for name, group in grouped:
    state = name[0]
    category = name[1]
    
    # Combine all the posts into one string
    posts = ' '.join(group['PostText'].tolist())
    
    # Tokenize the text into sentences
    sentences = sent_tokenize(posts)
    
    # Tokenize, stop, stem, and count the words in each sentence
    words = []
    for sentence in sentences:
        sentence_words = [stemmer.stem(word.lower()) for word in word_tokenize(sentence) if word.lower() not in stop_words and word.isalpha()]
        words.extend(sentence_words)
    
    # Count the frequency of each term
    frequency = Counter(words)
    
    # Get the top 10 terms
    top_terms = ', '.join([word for word, count in frequency.most_common(10)])
    
    # Get the top 10 questions
    questions = []
    for sentence in sentences:
        if '?' in sentence:
            sentence_questions = [question.strip() for question in sentence.split('?') if question.strip()]
            questions.extend(sentence_questions)
    top_questions = '\n'.join([question for question, count in Counter(questions).most_common(10)])
    
    # Add the results to the dataframe
    results_df = results_df.append({'State': state, 'Category': category, 'Top Terms': top_terms, 'Top Questions': top_questions}, ignore_index=True)

# Merge subcategories within each category
results_df['Category'] = results_df['Category'].str.split(' - ').str[0]
results_df = results_df.groupby(['State', 'Category']).agg({'Top Terms': ', '.join, 'Top Questions': '\n'.join}).reset_index()

In [ ]:
# Load the dataframe
df = test

# Tokenize and stem the text, and remove stop words and non-alpha tokens
def tokenize(text):
    tokens = [stemmer.stem(token.lower()) for token in word_tokenize(text) if token.isalpha() and token.lower() not in stop_words and len(token) > 1]
    return tokens

# Initialize the result dataframe
result = pd.DataFrame(columns=['State', 'Category', 'Top Terms', 'Top 3 Questions'])

# Loop through each state
for state in df['StateAbbr'].unique():
    # Filter the dataframe by state
    state_df = df[df['StateAbbr'] == state]
    
    # Loop through each category
    for category in state_df['Category'].unique():
        # Filter the dataframe by category
        category_df = state_df[state_df['Category'] == category]
        
        # Tokenize and stem the text
        tokens = [tokenize(text) for text in category_df['PostText']]
        
        # Flatten the list of tokens
        flat_tokens = [token for sublist in tokens for token in sublist]
        
        # Count the frequency of each token
        term_frequency = Counter(flat_tokens)
        
        # Get the top 10 terms
        top_terms = [term for term, count in term_frequency.most_common(10)]
        
        # Get the top 3 questions
        questions = []
        for text in category_df['PostText']:
            # Tokenize the text into sentences
            sentences = sent_tokenize(text)
            
            # Loop through each sentence
            for sentence in sentences:
                # Check if the sentence ends with a question mark
                if sentence.endswith('?'):
                    # Remove leading and trailing white space
                    sentence = sentence.strip()
                    
                    # Append the sentence to the list of questions
                    questions.append(sentence)
                    
                    # Break out of the loop if we have found 3 questions
                    if len(questions) == 5:
                        break
                        
            # Break out of the loop if we have found 3 questions
            if len(questions) == 5:
                break
        
        # Add the results to the result dataframe
        result = result.append({
            'State': state,
            'Category': category,
            'Top Terms': ', '.join(top_terms),
            'Top Question': '\n'.join(questions)
        }, ignore_index=True)

# Print the result dataframe

In [ ]:
result_sorted = result.sort_values(by="State", ascending=True)

In [ ]:
result_sorted.to_csv("Top_Question_Categorically.csv")